<a href="https://colab.research.google.com/github/vmartinezarias/Curso_Ecologia_Paisaje_y-Ecoacustica/blob/main/3%20-%20Calculo_Indices_satelitales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtención de índices Satelitales


## Importar las bibliotecas necesarias

In [ ]:
import ee
import geemap
import geopandas as gpd
from google.colab import drive

## Autenticar y inicializar Earth Engine, Montar drive y obtener área de estudio


In [ ]:
# Autenticar y inicializar Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-vmanuelmartinez')

# Montar Google Drive para acceder a los archivos
drive.mount('/content/drive')

# Cargar la región de interés (ROI) desde un archivo .shp usando geopandas
roi_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Area_Estudio.shp'
roi_gdf = gpd.read_file(roi_path)
roi = geemap.geopandas_to_ee(roi_gdf)

## Definir parámetros para búsqueda de imágenes de Sentinel-2A

El formato de fecha es YYYY-MM-DD
También es define el umbral (porcentaje) de nubes permitidas

In [ ]:
start_date = '2023-01-01'
end_date = '2023-07-31'
cloud_threshold = 10

## Obtener Sentinel-2
Aquí se usa quality mosaic para seleccionar los datos de Sentinel.

In [ ]:
# Crear colección de imágenes Sentinel-2 filtrada por fecha, ROI y porcentaje de nubes
S2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterDate(start_date, end_date)
      .filterBounds(roi)
      .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", cloud_threshold))

# Calcular NDVI y agregarlo como banda a la colección
S2_with_ndvi = S2.map(lambda image: image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')))
quality_mosaic = S2_with_ndvi.qualityMosaic('NDVI').clip(roi)
final_mosaic_rgb = quality_mosaic.select(["B4", "B3", "B2"])

# Configuraciones de visualización basadas en tus especificaciones en QGIS
vis_sentinel = {"min": 143, "max": 2166, "bands": ["B4", "B3", "B2"]}
vis_mndwi = {"min": -0.86564, "max": 0.332712, "palette": ['blue', 'white', 'green']}
vis_ndmi = {"min": -0.534972, "max": 0.620737, "palette": ['brown', 'white', 'green']}
vis_slavi = {"min": 0.2830355, "max": 2.1009374, "palette": ['lightyellow', 'darkgreen']}
vis_ndvi = {"min": -0.0801394, "max": 0.9122024, "palette": ['purple', 'blue', 'green', 'yellow']}
vis_brillo = {"min": 751.2558, "max": 13024.252, "palette": ['white', 'black']}

# Crear el mapa y añadir las capas configuradas
Map = geemap.Map()
Map.centerObject(roi, 10)
Map.addLayer(final_mosaic_rgb, vis_sentinel, "Sentinel2")

## Cálculo y exportación de índices espectrales
Consultar indexdatabase.de para información concernietne a los índices.
En este caso se calculan: NDVI, SLAVI, MNDWI, NDMI, Brillo.

El CRS es 32618.

Los índices son ubicados en el DRIVE.

In [ ]:
# Calcular y añadir los índices, asegurando el recorte al ROI
MNDWI = quality_mosaic.normalizedDifference(['B3', 'B11']).rename('MNDWI').clip(roi)
NDMI = quality_mosaic.normalizedDifference(['B8', 'B11']).rename('NDMI').clip(roi)
SLAVI = quality_mosaic.expression("(NIR) / (RE1 + SWIR2)", {
    "NIR": quality_mosaic.select('B8A'),
    "RE1": quality_mosaic.select('B5'),
    "SWIR2": quality_mosaic.select('B11')
}).rename('SLAVI').clip(roi)
NDVI = quality_mosaic.normalizedDifference(['B8', 'B4']).rename('NDVI').clip(roi)
Brillo = quality_mosaic.expression(
    "0.3037 * B + 0.2793 * G + 0.4743 * R + 0.5585 * NIR + 0.5082 * WV + 0.1863 * SWIR", {
        "B": quality_mosaic.select('B2'),
        "G": quality_mosaic.select('B3'),
        "R": quality_mosaic.select('B4'),
        "NIR": quality_mosaic.select('B8'),
        "WV": quality_mosaic.select('B9'),
        "SWIR": quality_mosaic.select('B11')
    }).rename('Brillo').clip(roi)

# Añadir las capas al mapa con las configuraciones de visualización
Map.addLayer(MNDWI, vis_mndwi, "MNDWI")
Map.addLayer(NDMI, vis_ndmi, "NDMI")
Map.addLayer(SLAVI, vis_slavi, "SLAVI")
Map.addLayer(NDVI, vis_ndvi, "NDVI")
Map.addLayer(Brillo, vis_brillo, "Brillo")

# Proyección deseada: EPSG:32618
projection = 'EPSG:32618'
scale = 10

# Exportar las imágenes a Google Drive, recortadas al ROI y con proyección EPSG:32618
exports = {
    "MNDWI": MNDWI,
    "NDMI": NDMI,
    "SLAVI": SLAVI,
    "NDVI": NDVI,
    "Brillo": Brillo,
    "Sentinel": final_mosaic_rgb
}

for name, image in exports.items():
    task = ee.batch.Export.image.toDrive(
        image=image.reproject(crs=projection, scale=scale),  # Aplicar la proyección deseada
        description=name,
        folder='Curso_Ecologia_Paisaje_Ecoacustica',
        fileNamePrefix=name,
        region=roi.geometry(),
        scale=scale,
        maxPixels=1e13
    )
    task.start()
    print(f"Exportando {name} a Google Drive.")

# Mostrar el mapa
Map
